### Fake news detection with AI algorithms

### Part 2

### Generate and Training IA model

In [3]:
#Libraries
#Read Libraries for the proyect
import pandas as pd
#EDA (Exploratory Data Analysis)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#AI Models
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


#Read Strings
import re
import string

In [4]:
#Read Data set news
df_news = pd.read_csv('../Data/news.csv')
#Clean column Unnamed
df_news.drop(["Unnamed: 0"], axis = 1, inplace = True)
df_news.head()

,Text,Type
0,WASHINGTON (Reuters) - Donald Trump on Monday ...,1
1,America is falling in love with Melania Trump....,0
2,BERNE (Reuters) - Neutral Switzerland is prepa...,1
3,A woman who was born in Nazi Germany says that...,0
4,"At the very beginning of May, Trump s Departme...",0


### Crear un clasificador para el texto

Es importante "standarizar" de una u otra manera la estructura del texto, por ello aplicamos un filtro que nos permita correjir ciertos aspectos de las cadenas.

In [5]:
#reset for text of the stings
def words( text ):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [6]:
#aplicar la función words al data frame
df_news["Text"] = df_news["Text"].apply(words)

Definimos la variable dependiente y la independiente

In [7]:
#variables
x = df_news["Text"]
y = df_news["Type"]

Definir el espacio de entrenamiento

In [8]:
   #Train test (train_test_split es una funcion predefinida en Scikit y declarada en la cabecera mediante el argumento import se encarga de dividir
    #el data set para el entrenamiento del modelo para este caso se reservara un 25% para evaluar el resultado mediante una variable llamada 
    #xv_test que sera la columna de noticias luego de vectorizarla.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25 )

In [9]:
#tarea: Revisar overfiting y tiempo de aprendizaje

Ahora Vamos a convertir las cadenas de texto en vectores por medio de Scikit

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

### Clasificador de Regresión Logistica 

Ahora aplicamos una herramienta de regresión logistica para clasificar las noticias 

In [11]:
LoR = LogisticRegression()
LoR.fit(xv_train, y_train) #Esta linea toma la vectorización efectuada a las cadenas de texto y usa la columna de 0 y 1 de Y_train para realizar el entrenamiento

LogisticRegression()

In [12]:
prediction_LoR = LoR.predict(xv_test) #esta linea introduce las noticias que se reservaron para la evaluación (test) y genera la predicción

In [13]:
LoR.score(xv_test, y_test) #arrojamos el accuracy estimado de la prediccion efectuada en la linea anterior

0.9861792242532322

In [14]:
#Desempeño del modelo de regresión Logistica
print(classification_report(y_test, prediction_LoR))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5877
           1       0.98      0.99      0.99      5338

    accuracy                           0.99     11215
   macro avg       0.99      0.99      0.99     11215
weighted avg       0.99      0.99      0.99     11215



Luego de aplicar el algoritmo de Regresión Logistica, vamos a implemetar otros y ver su desempeño, la lógica detrás de ls implementación es la misma... aprendizaje, testing, accuracy...

#### Gradient Boosting Classifier

In [15]:
GraBooCla = GradientBoostingClassifier(random_state=0)
GraBooCla.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [16]:
pred_gbc = GraBooCla.predict(xv_test)

In [17]:
GraBooCla.score(xv_test, y_test)

0.9948283548818546

In [18]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5877
           1       0.99      1.00      0.99      5338

    accuracy                           0.99     11215
   macro avg       0.99      0.99      0.99     11215
weighted avg       0.99      0.99      0.99     11215



### Clasificador tipo Árbol de Decisiones

In [19]:
ArD = DecisionTreeClassifier()
ArD.fit(xv_train, y_train)

DecisionTreeClassifier()

In [20]:
pred_ArD = ArD.predict(xv_test)

In [21]:
ArD.score(xv_test, y_test)

0.9946500222915737

In [22]:
print(classification_report(y_test, pred_ArD))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5877
           1       1.00      0.99      0.99      5338

    accuracy                           0.99     11215
   macro avg       0.99      0.99      0.99     11215
weighted avg       0.99      0.99      0.99     11215



### Clasificador Random Forest 

In [23]:
RFcl = RandomForestClassifier(random_state=0)
RFcl.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [24]:
pred_RFcl = RFcl.predict(xv_test)

In [25]:
RFcl.score(xv_test, y_test)

0.9860008916629514

In [26]:
print(classification_report(y_test, pred_RFcl))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5877
           1       0.99      0.98      0.99      5338

    accuracy                           0.99     11215
   macro avg       0.99      0.99      0.99     11215
weighted avg       0.99      0.99      0.99     11215



### Testeando el Modelo 

Luego de tener los modelos entrenados, vamos a validar la herramienta que acabamos de crear por medio de una función que reciba como entrada una noticia, la valide, luego de ello la vectorice, a continuación, la clasifique mediante las 3 herramientas y finalmente arroje una clasificación para la entrada.

In [27]:
testing_news = pd.DataFrame({"Text":[" KILLED: Rancher and protest spokesman Robert  Lavoy  Finicum (Image Source: RT International)21st Century Wire says New eye witness testimony indicates that the FBI and other federal agencies were never fired upon, but have shot and killed leading protester, Robert  Lavoy  Finicum, and wounding one other in a elaborate operation that took place along a rural highway near Burns, Oregon.For 26 days since the protest and occupation began at the Malheur Wildlife Refuge outside of Burns, Oregon there were no one was hurt and there were no injuries. That all changed early Tuesday evening when the FBI and Oregon State Police raised the ante, bringing the 4 week standoff to a dramatic ending.As detailed in yesterday s report, according to 18 yr old passenger and eyewitness, Victoria Sharp, dozens of federal vehicles and armed agents had set-up a pre-positioned roadblocks along an isolated stretch of rural Highway 395 before intercepting two vehicles and firing upon one of them, killing rancher and protest spokesman, Robert  Lavoy  Finicum, from Chino Valley, Arizona, and wounding Ryan Bundy of Bunkerville, Nevada.A total of 8 persons were detained, 5 of whom were charged with conspiracy to impede federal officers and are waiting arraignment.21WIRE journalist Patrick Henningsen weighs in on the situation in Oregon and discusses its implications with news anchor Rory Suchet from RT International. Watch: UPDATE 1-28-2016 at 11:30PM ET: Due to public pressure and intense speculation about the events of Tuesday evening, and because of the YouTube recording of 18 yr old eye witness Victoria Sharp, the FBI has released the unedited aerial video footage from Tuesday evening s incident which took place along Highway 395 here. According to the official FBI statement:  We feel that it is necessary to show the whole thing unedited in the interest of transparency.  The FBI video entitled,  Complete, Unedited Video of Joint FBI and OSP Operation 01/26/2016 , does show the victim Lavoy Finicum exiting the truck awkwardly in at least two feet of snow, and he clearly does not  charge towards  the SWAT team, and his hands are clearly held high above his head as he exits his truck when confronted by a SWAT team, before being shot multiple times by marksmen and falling in the snow. Also, the scale and size of this operation is very evident by the footage, and the apprehension of the protesters was not the result of a mere  traffic stop  as it was wrongly characterized in multiple mainstream media reports. On its own, this FBI drone footage would appear to support previous government and media claims by CNN, Washington Post and others that the victim was indeed  reaching towards his waistband  which gave federal and state police justification to unleash deadly force. However, it is still not completely clear from this aerial footage whether or not Finicum was holstered (he is said to normally carry a gun on the right hip, but FBI are claiming he had a 9mm gun in his inside left breast pocket), but also whether or not he lowered his arms before, or after he was shot multiple times. For example, if Finicum had his hands in the air and was then was shot in the abdomen first, then any man s natural reaction would be to then lower his hand(s) and clutch the wound. Additionally, if any shots were fired, even in the vicinity, then confusion could have ensued which might have prompted Finicum to reach for his weapon. Either way, it is impossible to make any such a forensic determination without a corresponding audio track  which would help to determine if any shots were fired before Finicum s hand(s) could be seen lowering.WARNING: The following images depict violence and death which some readers may find disturbing. Watch: SEE MORE OREGON STANDOFF NEWS AT: 21st Century Wire Oregon Standoff Fil"]})

In [28]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "True News"
    
def manual_testing(news):
    testing_news = {"Text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["Text"] = new_def_test["Text"].apply(words) 
    new_x_test = new_def_test["Text"]
    new_xv_test = vectorization.transform(new_x_test)
    prediction_LoR = LoR.predict(new_xv_test)
    pred_ArD = ArD.predict(new_xv_test)
    pred_gbc = GraBooCla.predict(new_xv_test)
    pred_RFcl = RFcl.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(prediction_LoR[0]), output_lable(pred_ArD[0]), output_lable(pred_gbc[0]), output_lable(pred_RFcl[0])))

In [ ]:
#Ingrese a cualquier portal de noticias en ingles y copie una noticia política u económica  y péguela aquí. El clasificador le dirá si es falsa 
news = str(input())
manual_testing(news)

In [ ]:
#Ingrese a cualquier portal de noticias en ingles y copie una noticia política u económica  y péguela aquí. El clasificador le dirá si es falsa
news = str(input())
manual_testing(news)